In [ ]:
import torch

from transformers import AutoConfig, AutoTokenizer, LlamaForCausalLM
from model import DiscreteDiffusionModel, get_anneal_attn_mask


/home/exx/miniconda3/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Use provided 4d attn-mask


In [2]:
id_model = 'diffusionfamily/diffullama'
device = 'cuda:0'

tokenizer = AutoTokenizer.from_pretrained(id_model)
config = AutoConfig.from_pretrained(id_model)
model = LlamaForCausalLM.from_pretrained(
    id_model,
    torch_dtype=torch.bfloat16
).eval()

model = DiscreteDiffusionModel(
    model=model,
    config=config,
    tokenizer=tokenizer,
    device=device
).eval().to(device)

Loading checkpoint shards: 100%|██████████| 3/3 [00:00<00:00,  7.57it/s]


In [ ]:
x = tokenizer.encode("good morning my friend", return_tensors='pt')
attention_mask = get_anneal_attn_mask(seq_len, batch_size, dtype=x_embed.dtype, device=x.device, attn_mask_ratio=1.0) # all 0

In [ ]:
model()

tensor([[   1, 1781, 7250,  590, 5121]])